# Ejercicio 5. Creación de un triangulo run-off de pagos

El primer paso consiste en importar los datos (10.000 siniestros ficticios desde 2013 al 2018) y mostramos los primeros registros:

In [1]:
import pandas as pd
import numpy as np

data = pd.read_csv('Siniestros.csv',sep=';')
data.head()

,Siniestro,Poliza,Fec_ocu,Fec_com,Fec_pago,Importe
0,1,31082,12-09-2016,21-09-2016,03-07-2017,83
1,2,61275,20-11-2017,03-01-2018,02-02-2018,71
2,3,72257,10-06-2013,25-06-2013,15-07-2013,115
3,4,73111,17-09-2013,05-10-2013,20-09-2018,67
4,5,84755,30-06-2017,12-07-2017,30-09-2017,122


Una vez importados los datos, modificamos los campos de fecha con el formato de fecha adecuado.

*Si formato original no fuera YYYY-MM-DD utilizar pd.to_datetime(data['Accident_Date'], format='%d-%m-%Y')*

In [2]:
data['Fec_ocu'] = pd.to_datetime(data['Fec_ocu'], format='%d-%m-%Y')
data['Fec_pago'] = pd.to_datetime(data['Fec_pago'], format='%d-%m-%Y')
data.head()

,Siniestro,Poliza,Fec_ocu,Fec_com,Fec_pago,Importe
0,1,31082,2016-09-12,21-09-2016,2017-07-03,83
1,2,61275,2017-11-20,03-01-2018,2018-02-02,71
2,3,72257,2013-06-10,25-06-2013,2013-07-15,115
3,4,73111,2013-09-17,05-10-2013,2018-09-20,67
4,5,84755,2017-06-30,12-07-2017,2017-09-30,122


Acto seguido, bien creamos nuevos campos con el año (ejemplo *Accident_Year*) o, como en este caso, modificamos dichos campos convertiendolos a una periocidad anual (utilizo la variable *freq* para posteriormente modificar la periodicidad facilmente).

In [3]:
freq='Y' 
data['Fec_ocu'] = data['Fec_ocu'].dt.to_period(freq=freq)
data['Fec_pago'] = data['Fec_pago'].dt.to_period(freq=freq)
data.head()

,Siniestro,Poliza,Fec_ocu,Fec_com,Fec_pago,Importe
0,1,31082,2016,21-09-2016,2017,83
1,2,61275,2017,03-01-2018,2018,71
2,3,72257,2013,25-06-2013,2013,115
3,4,73111,2013,05-10-2013,2018,67
4,5,84755,2017,12-07-2017,2017,122


Calculamos un nuevo campo llamado ‘Dev’ con el periodo de tiempo transcurrido (o desarrollo) como la diferencia entre ambos periodos (en este caso años) restandolos como números enteros: .view(int).
_Nota: el modo de restar dos fechas ha sido modificado en las últimas versiones (este ejercicio funciona en la version 1.3.5, en versiones anteriores de pandas se utilizaba .astype(int)_

In [4]:
#version posterior a 1.3.5: data['Dev'] = data['Fec_pago'].view(int) - data['Fec_ocu'].view(int)
#data['Dev'] = data['Fec_pago'].astype(int) - data['Fec_ocu'].astype(int)
data['Dev'] = data['Fec_pago'].astype(int) - data['Fec_ocu'].astype(int)
data.head()

<ipython-input-4-03d73ff7d391>:3: FutureWarning: The behavior of .astype from period[A-DEC] to int32 is deprecated. In a future version, this astype will return exactly the specified dtype instead of int64, and will raise if that conversion overflows.
  data['Dev'] = data['Fec_pago'].astype(int) - data['Fec_ocu'].astype(int)


,Siniestro,Poliza,Fec_ocu,Fec_com,Fec_pago,Importe,Dev
0,1,31082,2016,21-09-2016,2017,83,1
1,2,61275,2017,03-01-2018,2018,71,1
2,3,72257,2013,25-06-2013,2013,115,0
3,4,73111,2013,05-10-2013,2018,67,5
4,5,84755,2017,12-07-2017,2017,122,0


Agrupamos por año de accidente y año de desarrollo usando groupby(), y sumamos las cantidades de los siniestros:

In [5]:
datagg = data.groupby(['Fec_ocu','Dev']).agg({'Importe':'sum'}).reset_index()
datagg.head()

,Fec_ocu,Dev,Importe
0,2013,0,118066
1,2013,1,25877
2,2013,2,12392
3,2013,3,9743
4,2013,4,8599


Y una vez agrupados, guardamos los datos en una tabla 2-D (pivot table) basada en estas dos variables:

In [6]:
triangle = datagg.pivot(index='Fec_ocu',columns='Dev',values='Importe')
triangle

Dev,0,1,2,3,4,5
Fec_ocu,,,,,,
2013,118066.0,25877.0,12392.0,9743.0,8599.0,8977.0
2014,119961.0,26543.0,13813.0,11509.0,10339.0,NaN
2015,108140.0,27975.0,18551.0,14250.0,NaN,NaN
2016,107660.0,34826.0,19814.0,NaN,NaN,NaN
2017,112042.0,51641.0,NaN,NaN,NaN,NaN
2018,133511.0,NaN,NaN,NaN,NaN,NaN


In [7]:
accumulate = triangle.cumsum(axis=1)
accumulate

Dev,0,1,2,3,4,5
Fec_ocu,,,,,,
2013,118066.0,143943.0,156335.0,166078.0,174677.0,183654.0
2014,119961.0,146504.0,160317.0,171826.0,182165.0,NaN
2015,108140.0,136115.0,154666.0,168916.0,NaN,NaN
2016,107660.0,142486.0,162300.0,NaN,NaN,NaN
2017,112042.0,163683.0,NaN,NaN,NaN,NaN
2018,133511.0,NaN,NaN,NaN,NaN,NaN


Exportamos a excel:

In [8]:
accumulate.to_excel('triangulo_acumulado.xlsx')